In [9]:
%load_ext autoreload
%autoreload 2

from pm4py.objects.dcr.exporter import exporter
from pm4py.algo.evaluation.dcr.algorithm import *
from pm4py.algo.discovery.dcr_discover import algorithm
from pm4py.objects.dcr.obj import dcr_template

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Collect datasets

In [10]:
# all bpi + sepsis + rtfmp + dreyers
logs_list = { 'Dreyers':'/home/vco/Datasets/Dreyers Foundation.xes',
              'BPIC19' :'/home/vco/Datasets/BPI_Challenge_2019.xes'}
logs_folder = '/home/vco/Datasets/data/TKDE_Benchmark'
for file in os.listdir(logs_folder):
    if file.endswith(".xes"):
        name = os.path.basename(file.split('.')[0])
        logs_list[name] = os.path.join(logs_folder, file)
print(f'[i] Started benchmarking on {len(logs_list)} logs')
print(logs_list)

[i] Started benchmarking on 14 logs
{'Dreyers': '/home/vco/Datasets/Dreyers Foundation.xes', 'BPIC19': '/home/vco/Datasets/BPI_Challenge_2019.xes', 'RTFMP': '/home/vco/Datasets/data/TKDE_Benchmark/RTFMP.xes', 'BPIC17_f': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC17_f.xes', 'BPIC13_i': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC13_i.xes', 'BPIC15_1f': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC15_1f.xes', 'BPIC15_3f': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC15_3f.xes', 'BPIC15_4f': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC15_4f.xes', 'BPIC14_f': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC14_f.xes', 'BPIC15_5f': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC15_5f.xes', 'BPIC12': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC12.xes', 'BPIC15_2f': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC15_2f.xes', 'SEPSIS': '/home/vco/Datasets/data/TKDE_Benchmark/SEPSIS.xes', 'BPIC13_cp': '/home/vco/Datasets/data/TKDE_Benchmark/BPIC13_cp.xes'}


In [11]:
# all pdc


# Amines metrics
https://www.sciencedirect.com/science/article/pii/S0957417423014264

In [12]:
G = {
    'events': set(['Co','Wr','Ch','Ap','Su','Do','Ex','No']),
    'conditionsFor': {'Wr':{'Co'}},
    'milestonesFor': {'Su':{'Wr'}},
    'responseTo': {'Co':{'Wr'},'Ex':{'No'}},
    'noResponseTo': {'Ap':{'Wr'}},
    'includesTo': {'Wr':{'Ch'}},
    'excludesTo': {'Su':{'Su'}},
    'marking': {'executed': set(['Do']),
                'included': set(['Co','Wr','Ap','Su','Do','Ex','No']),
                'pending': set(['Wr']),
                'executedTime': {},  # Gives the time since a event was executed
                'pendingDeadline': {}  # The deadline until an event must be executed
                },
    'conditionsForDelays': {},
    'responseToDeadlines': {},
    'subprocesses': {},
    'nestings': {},
    'labels': set(),
    'labelMapping': {},
    'roles': set(),
    'roleAssignments': {},
    'readRoleAssignments': {}
}
print(f'[i] Size: {dcr_size(G)}')
print(f'[i] Density: {dcr_density(G)}')
print(f'[i] Separability: {dcr_separability(G)}')
print(f'[i] Constraint Variability: {dcr_constraint_variablity_metric(G)}')
print(run_all_dcr_metrics(G))

[i] Size: 15
[i] Density: 1.2
[i] Separability: 0.2
[i] Constraint Variability: 0.9999999999999999
{'size': 15, 'density': 1.2, 'separability': 0.2, 'constraint_variability': 0.9999999999999999}


/home/vco/.pyenv/versions/3.11.0/envs/pm4py-dcr/lib/python3.11/site-packages/numpy/lib/scimath.py:386: RuntimeWarning: divide by zero encountered in log
  return nx.log(x)/nx.log(n)
/home/vco/Projects/pm4py-dcr/pm4py/algo/evaluation/dcr/algorithm.py:85: RuntimeWarning: invalid value encountered in scalar multiply
  cv_sum += pct[rel.value] * np.emath.logn(rels_in_comp, pct[rel.value])
/home/vco/.pyenv/versions/3.11.0/envs/pm4py-dcr/lib/python3.11/site-packages/numpy/lib/scimath.py:386: RuntimeWarning: invalid value encountered in scalar divide
  return nx.log(x)/nx.log(n)


In [13]:
G = {
    'events': set(['A','B','C','D','E','F']),
    'conditionsFor': {'A':{'B'},'C':{'B'}},
    'milestonesFor': {},
    'responseTo': {},
    'noResponseTo': {},
    'includesTo': {'A':{'D','E','F'},'B':{'D','E','F'}},
    'excludesTo': {'C':{'D','E','F'}},
    'marking': {'executed': set(),
                'included': set(['A','B','C','D','E','F']),
                'pending': set(),
                'executedTime': {},  # Gives the time since a event was executed
                'pendingDeadline': {}  # The deadline until an event must be executed
                },
    'conditionsForDelays': {},
    'responseToDeadlines': {},
    'subprocesses': {},
    'nestings': {},
    'labels': set(),
    'labelMapping': {},
    'roles': set(),
    'roleAssignments': {},
    'readRoleAssignments': {}
}

# Based on pseudocode

In [14]:
def encode(G):
    enc = {}
    for e in G['events']:
        enc[e] = set()
    for e in G['events']:
        for e_prime in G['events']:
            for rel in Relations:
                if e in G[rel.value] and e_prime in G[rel.value][e]:
                    if rel in [Relations.C,Relations.M]:
                        enc[e].add((e_prime,rel.value,'in'))
                    else:
                        enc[e].add((e_prime,rel.value,'out'))
                if e_prime in G[rel.value] and e in G[rel.value][e_prime]:
                    if rel in [Relations.C,Relations.M]:
                        enc[e].add((e_prime,rel.value,'out'))
                    else:
                        enc[e].add((e_prime,rel.value,'in'))
    return enc

def getOppositeRelDictStr(relStr,direction, event, nestingId):
    relationDictStrDel = (event,relStr,"out" if direction == "in" else "in")
    relationDictStrAdd = (nestingId,relStr,"out"  if direction == "in" else "in")
    
    return relationDictStrDel, relationDictStrAdd


def nest(enc, events_source, nest_id = 0):
    nesting_map = {}
    nesting_ids = set()
    events = deepcopy(events_source)
    while True:
        cands = {}
        for e in events:
            for j in events:
                arrow_s = frozenset(enc[e].intersection(enc[j]))
                if len(arrow_s)>0:
                    cands[arrow_s] = set([e, j])
        
        for arrow_s in cands.keys():
            for e in G['events']:
                if len(arrow_s)>0 and arrow_s.issubset(enc[e]):
                    cands[arrow_s].add(e)
        
        best_score = 0
        best = None
        for arrow_s in cands.keys():
            cand_score = (len(cands[arrow_s])-1)*len(arrow_s)
            if cand_score>best_score:
                best_score = cand_score
                best = arrow_s
                
        if best_score == 0:
            break
        print(f'     [events] {events} \n'
              f'[cands[best]] {cands[best]} \n'
              f'       [best] {best} \n'
              f'        [enc] {enc} \n '
              f'      [cands] {cands} \n')
        nest_id += 1
        nesting_ids.add(f'N{nest_id}')
        enc[f'N{nest_id}'] = set(best)
        for e in cands[best]:
            nesting_map[e] = f'N{nest_id}'
            enc[e] = enc[e].difference(best)
            for (e_prime,rel,direction) in best:
                opRelDel, opRelAdd = getOppositeRelDictStr(rel,direction,e,f'N{nest_id}')
                enc[e_prime].remove(opRelDel)
                enc[e_prime].add(opRelAdd)
        
        # nestings inside nestings
        found = True
        temp_events = deepcopy(cands[best])
        while found:
            enc_prime, nesting_ids_prime, nesting_map_prime = nest(enc, temp_events, nest_id) # replace with find_largest_nesting()
            if len(nesting_ids_prime)>0:
                enc = enc_prime
                nesting_ids = nesting_ids.union(nesting_ids_prime)
                nesting_map = nesting_map | nesting_map_prime
                temp_events = temp_events.difference(cands[best])
            else:
                found = False
            
        events = events.difference(cands[best])
        enc_prime, nesting_ids_prime, nesting_map_prime = nest(enc, events, nest_id) # replace with find_largest_nesting()
        enc = enc_prime
        nesting_ids = nesting_ids.union(nesting_ids_prime)
        # print(f'[nesting_map] {nesting_map}')
        # print(f'[nesting_map_prime] {nesting_map_prime}')
        nesting_map = nesting_map | nesting_map_prime
        # add a second while loop that recursively calls nest()
    return enc, nesting_ids, nesting_map


nest_id = 0
enc = encode(G)
events = G['events']
enc, nesting_ids, nesting_map = nest(enc, events)
print(nesting_ids,nesting_map)

     [events] {'A', 'F', 'E', 'B', 'D', 'C'} 
[cands[best]] {'F', 'E', 'D'} 
       [best] frozenset({('A', 'includesTo', 'in'), ('B', 'includesTo', 'in'), ('C', 'excludesTo', 'in')}) 
        [enc] {'A': {('B', 'conditionsFor', 'in'), ('D', 'includesTo', 'out'), ('E', 'includesTo', 'out'), ('F', 'includesTo', 'out')}, 'F': {('A', 'includesTo', 'in'), ('B', 'includesTo', 'in'), ('C', 'excludesTo', 'in')}, 'E': {('A', 'includesTo', 'in'), ('B', 'includesTo', 'in'), ('C', 'excludesTo', 'in')}, 'B': {('F', 'includesTo', 'out'), ('C', 'conditionsFor', 'out'), ('E', 'includesTo', 'out'), ('D', 'includesTo', 'out'), ('A', 'conditionsFor', 'out')}, 'D': {('A', 'includesTo', 'in'), ('B', 'includesTo', 'in'), ('C', 'excludesTo', 'in')}, 'C': {('B', 'conditionsFor', 'in'), ('F', 'excludesTo', 'out'), ('D', 'excludesTo', 'out'), ('E', 'excludesTo', 'out')}} 
       [cands] {frozenset({('B', 'conditionsFor', 'in'), ('D', 'includesTo', 'out'), ('E', 'includesTo', 'out'), ('F', 'includesTo', 'out')}

# Object oriented code

In [15]:
class Nesting(object):

    def __init__(self):
        self.nesting_ids = set()
        self.nesting_map = {}
        self.nest_id = 0
        self.enc = None
        self.in_rec_step = 0
        self.out_rec_step = 0
        self.debug = False
        
    def create_encoding(self,dcr_graph):
        self.enc = encode(dcr_graph)

    def find_largest_nesting(self, events_source, parent_nesting=None, inrec=False):
        cands = {}
        events = deepcopy(events_source)
        for e in events:
            for j in events:
                arrow_s = frozenset(self.enc[e].intersection(self.enc[j]))
                if len(arrow_s)>0:
                    cands[arrow_s] = set([e, j])
        
        for arrow_s in cands.keys():
            for e in events:
                if len(arrow_s)>0 and arrow_s.issubset(self.enc[e]):
                    cands[arrow_s].add(e)
        
        best_score = 0
        best = None
        for arrow_s in cands.keys():
            cand_score = (len(cands[arrow_s])-1)*len(arrow_s)
            if cand_score>best_score:
                best_score = cand_score
                best = arrow_s
        if best and len(cands[best])>1 and len(best)>= 1:
            if self.debug:
                print(f'[out]:{self.out_rec_step} [in]:{self.in_rec_step} \n'
                      f'     [events] {events} \n'
                      f'[cands[best]] {cands[best]} \n'
                      f'       [best] {best} \n'
                      f'        [enc] {enc} \n '
                      f'      [cands] {cands} \n')
            
            self.nest_id += 1
            self.nesting_ids.add(f'N{self.nest_id}')
            self.enc[f'N{self.nest_id}'] = set(best)
            if parent_nesting:
                parent_nesting['events'] = parent_nesting['events'].difference(cands[best])
                parent_nesting['events'].add(f'N{self.nest_id}')
                self.nesting_map[f'N{self.nest_id}'] = parent_nesting['id']
            
            for e in cands[best]:
                self.nesting_map[e] = f'N{self.nest_id}'
                self.enc[e] = self.enc[e].difference(best)
                for (e_prime,rel,direction) in best:
                    opRelDel, opRelAdd = getOppositeRelDictStr(rel,direction,e,f'N{self.nest_id}')
                    self.enc[e_prime].discard(opRelDel)#.remove(opRelDel)
                    self.enc[e_prime].add(opRelAdd)
            
            self.enc[f'N{self.nest_id}'] = set(best)
            
            retval = [{'nestingEvents': cands[best], 'sharedRels': best}]
            found = True
            while found:
                temp_retval = self.find_largest_nesting(events_source=cands[best], parent_nesting={'id': f'N{self.nest_id}','events': cands[best]}, inrec=True)
                if temp_retval and len(temp_retval)>0:
                    retval.extend(temp_retval)
                    for tmp in temp_retval:
                        events = events.difference(tmp['nestingEvents'])
                else:
                    found = False
                self.in_rec_step += 1
            return retval
    
    def nest(self, events_source):
        nestingsArr = [{'nestingEvents': set(), 'sharedRels': set()}]
        events = deepcopy(events_source)
        while True:
            temp_retval = self.find_largest_nesting(events)
            if temp_retval and len(temp_retval)>0:
                nestingsArr.extend(temp_retval)
                for tmp in temp_retval:
                    events = events.difference(tmp['nestingEvents'])
            else:
                break
            self.out_rec_step += 1
        return self.nesting_map, self.nesting_ids

    def get_nested_dcr_graph(self):
        res_dcr = deepcopy(dcr_template)
        events = set(self.enc.keys())
        res_dcr['events'] = events
        res_dcr['marking']['included'] = events
        
        for n in self.nesting_ids:
            res_dcr['nestings'][n] = set()
        for k,v in self.nesting_map.items():
            res_dcr['nestings'][v].add(k)
        
        def shouldAdd(rel,direction):
            return direction == 'in' if rel in [Relations.C.value,Relations.M.value] else direction == 'out'
        
        for e, v in self.enc.items():
            for e_prime, rel, direction in v:
                if shouldAdd(rel,direction):
                    if e not in res_dcr[rel]:
                        res_dcr[rel][e] = set()
                    res_dcr[rel][e].add(e_prime)
        return res_dcr

In [16]:
n = Nesting()  
n.create_encoding(G)
n.nest(G['events'])

({'F': 'N1', 'E': 'N1', 'D': 'N1', 'B': 'N2', 'A': 'N2'}, {'N1', 'N2'})

In [17]:
n.get_nested_dcr_graph()

{'events': {'A', 'B', 'C', 'D', 'E', 'F', 'N1', 'N2'},
 'conditionsFor': {'A': {'B'}, 'C': {'B'}},
 'milestonesFor': {},
 'responseTo': {},
 'noResponseTo': {},
 'includesTo': {'N2': {'N1'}},
 'excludesTo': {'C': {'N1'}},
 'marking': {'executed': set(),
  'included': {'A', 'B', 'C', 'D', 'E', 'F', 'N1', 'N2'},
  'pending': set(),
  'executedTime': {},
  'pendingDeadline': {}},
 'conditionsForDelays': {},
 'responseToDeadlines': {},
 'subprocesses': {},
 'nestings': {'N1': {'D', 'E', 'F'}, 'N2': {'A', 'B'}},
 'nestingsMap': {},
 'labels': set(),
 'labelMapping': {},
 'roles': set(),
 'roleAssignments': {},
 'readRoleAssignments': {}}

In [35]:
n.nesting_map

{'Payment': 'N1',
 'Send Appeal to Prefecture': 'N1',
 'Insert Date Appeal to Prefecture': 'N1',
 'Appeal to Judge': 'N1',
 'Send Fine': 'N1',
 'Insert Fine Notification': 'N2',
 'Receive Result Appeal from Prefecture': 'N2',
 'Notify Result Appeal to Offender': 'N3',
 'Add penalty': 'N3'}

In [19]:
G_copy = deepcopy(G)

In [20]:
import pm4py

In [21]:
sepsis_log = pm4py.read_xes('/home/vco/Datasets/Sepsis Cases - Event Log.xes',return_legacy_log_object=True)
sepsis_reference_dcr, _ = algorithm.apply(sepsis_log)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

[i] Mining with basic DisCoveR


In [40]:
n = Nesting()
n.create_encoding(sepsis_reference_dcr)
n.nest(sepsis_reference_dcr['events'])
nested_sepsis = n.get_nested_dcr_graph()

In [41]:
nested_sepsis

{'events': {'Admission IC',
  'Admission NC',
  'CRP',
  'ER Registration',
  'ER Sepsis Triage',
  'ER Triage',
  'IV Antibiotics',
  'IV Liquid',
  'LacticAcid',
  'Leucocytes',
  'N1',
  'N2',
  'N3',
  'N4',
  'N5',
  'N6',
  'Release A',
  'Release B',
  'Release C',
  'Release D',
  'Release E',
  'Return ER'},
 'conditionsFor': {'IV Antibiotics': {'ER Sepsis Triage'},
  'Return ER': {'N2', 'N3'},
  'Release B': {'ER Registration', 'ER Triage'},
  'Admission NC': {'ER Registration', 'ER Triage'},
  'N1': {'ER Sepsis Triage', 'N5'},
  'N2': {'Admission NC'},
  'N3': {'ER Registration', 'ER Triage'}},
 'milestonesFor': {},
 'responseTo': {},
 'noResponseTo': {},
 'includesTo': {},
 'excludesTo': {'IV Antibiotics': {'IV Antibiotics'},
  'Release A': {'Admission NC', 'N1', 'N3'},
  'ER Registration': {'ER Registration'},
  'IV Liquid': {'IV Liquid'},
  'Return ER': {'Admission IC', 'ER Triage', 'N6', 'Release B'},
  'ER Sepsis Triage': {'ER Sepsis Triage'},
  'Release B': {'N1', 'Ret

In [24]:
exporter.apply(nested_sepsis,'/home/vco/Projects/pm4py-dcr/models/sepsis_nested.xml')

In [25]:
rtfmp_log = pm4py.read_xes('/home/vco/Projects/pm4py-dcr/datasets/rtfmp.xes',return_legacy_log_object=True)
rtfmp_reference_dcr, _ = algorithm.apply(rtfmp_log)

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

[i] Mining with basic DisCoveR


In [36]:
bpic17_offer = {'events': {'O_Accepted',
  'O_Cancelled',
  'O_Create Offer',
  'O_Created',
  'O_Refused',
  'O_Returned',
  'O_Sent (mail and online)',
  'O_Sent (online only)'},
 'conditionsFor': {'O_Returned': {'O_Sent (mail and online)',
   'O_Sent (online only)'},
  'O_Accepted': {'O_Returned'},
  'O_Sent (online only)': {'O_Created'},
  'O_Sent (mail and online)': {'O_Created'},
  'O_Cancelled': {'O_Created'},
  'O_Created': {'O_Create Offer'},
  'O_Refused': {'O_Created'},
  'O_Create Offer': set()},
 'milestonesFor': {},
 'responseTo': {
  'O_Create Offer': {'O_Created'}},
 'noResponseTo': {},
 'includesTo': {},
 'excludesTo': {'O_Returned': {'O_Returned'},
  'O_Accepted': {'O_Accepted', 'O_Cancelled', 'O_Refused'},
  'O_Sent (online only)': {'O_Sent (mail and online)', 'O_Sent (online only)'},
  'O_Sent (mail and online)': {'O_Sent (mail and online)',
   'O_Sent (online only)'},
  'O_Cancelled': {'O_Accepted', 'O_Cancelled', 'O_Refused'},
  'O_Created': {'O_Created'},
  'O_Refused': {'O_Accepted', 'O_Cancelled', 'O_Refused'},
  'O_Create Offer': {'O_Create Offer'}},
 'marking': {'executed': set(),
  'included': {'O_Accepted',
   'O_Cancelled',
   'O_Create Offer',
   'O_Created',
   'O_Refused',
   'O_Returned',
   'O_Sent (mail and online)',
   'O_Sent (online only)'},
  'pending': set(),
  'executedTime': {},
  'pendingDeadline': {}},
 'conditionsForDelays': {},
 'responseToDeadlines': {},
 'subprocesses': {},
 'nestings': {},
 'nestingsMap': {},
 'labels': set(),
 'labelMapping': {},
 'roles': set(),
 'roleAssignments': {},
 'readRoleAssignments': {}}

In [42]:
n = Nesting()
n.create_encoding(bpic17_offer)
n.nest(bpic17_offer['events'])
nested_bpicoffer = n.get_nested_dcr_graph()

In [43]:
nested_bpicoffer

{'events': {'N1',
  'N2',
  'N3',
  'N4',
  'N5',
  'O_Accepted',
  'O_Cancelled',
  'O_Create Offer',
  'O_Created',
  'O_Refused',
  'O_Returned',
  'O_Sent (mail and online)',
  'O_Sent (online only)'},
 'conditionsFor': {'O_Returned': {'N4'},
  'O_Created': {'O_Create Offer'},
  'O_Accepted': {'O_Returned'},
  'N3': {'O_Created'},
  'N4': {'O_Created'}},
 'milestonesFor': {},
 'responseTo': {'O_Create Offer': {'O_Created'}},
 'noResponseTo': {},
 'includesTo': {},
 'excludesTo': {'O_Returned': {'O_Returned'},
  'O_Created': {'O_Created'},
  'O_Create Offer': {'O_Create Offer'},
  'N1': {'N2'},
  'N2': {'N1'},
  'N4': {'N5'},
  'N5': {'N4'}},
 'marking': {'executed': set(),
  'included': {'N1',
   'N2',
   'N3',
   'N4',
   'N5',
   'O_Accepted',
   'O_Cancelled',
   'O_Create Offer',
   'O_Created',
   'O_Refused',
   'O_Returned',
   'O_Sent (mail and online)',
   'O_Sent (online only)'},
  'pending': set(),
  'executedTime': {},
  'pendingDeadline': {}},
 'conditionsForDelays': {

In [26]:
exporter.apply(rtfmp_reference_dcr,'/home/vco/Projects/pm4py-dcr/models/rtfmp.xml')

In [27]:
n = Nesting()
n.create_encoding(rtfmp_reference_dcr)
n.nest(rtfmp_reference_dcr['events'])
nested_rtfmp = n.get_nested_dcr_graph()

In [28]:
nested_rtfmp

{'events': {'Add penalty',
  'Appeal to Judge',
  'Create Fine',
  'Insert Date Appeal to Prefecture',
  'Insert Fine Notification',
  'N1',
  'N2',
  'N3',
  'Notify Result Appeal to Offender',
  'Payment',
  'Receive Result Appeal from Prefecture',
  'Send Appeal to Prefecture',
  'Send Fine',
  'Send for Credit Collection'},
 'conditionsFor': {'Send for Credit Collection': {'Add penalty'},
  'N1': {'Create Fine'},
  'N2': {'Send Fine'},
  'N3': {'Insert Fine Notification'}},
 'milestonesFor': {},
 'responseTo': {'Insert Fine Notification': {'Add penalty'}},
 'noResponseTo': {},
 'includesTo': {},
 'excludesTo': {'Receive Result Appeal from Prefecture': {'Receive Result Appeal from Prefecture'},
  'Add penalty': {'Add penalty'},
  'Insert Fine Notification': {'Insert Fine Notification'},
  'Notify Result Appeal to Offender': {'Notify Result Appeal to Offender'},
  'Send Appeal to Prefecture': {'Send Appeal to Prefecture'},
  'Create Fine': {'Create Fine'},
  'Insert Date Appeal to Pr

In [29]:
exporter.apply(nested_rtfmp,'/home/vco/Projects/pm4py-dcr/models/rtfmp_nested.xml')

In [30]:
common_arcs_res = {}

cands = {}

for e in G['events']:
    common_arcs_res[e] = {}
    for j in G['events']: 
        common_arcs = enc[e].intersection(enc[j])
        if len(common_arcs)>0 and e != j:
            print(f'{e} {j} :{common_arcs}')
            common_arcs_res[e][j] = common_arcs
            if frozenset(common_arcs) not in cands:
                cands[frozenset(common_arcs)] = set()
            cands[frozenset(common_arcs)].add(e)
            cands[frozenset(common_arcs)].add(j)
            

In [31]:
cands

{}

# From colab

In [32]:
G = {
    'events': set(['A','B','C','D','E','F', 'G']),
    'conditionsFor': {'A':{'B'},'C':{'B'}},
    'milestonesFor': {'G': {'D', 'E'}},
    'responseTo': {},
    'noResponseTo': {},
    'includesTo': {'A':{'D','E','F'},'B':{'D','E','F'}},
    'excludesTo': {'C':{'D','E','F'}},
    'marking': {'executed': set(),
                'included': set(['A','B','C','D','E','F']),
                'pending': set(),
                'executedTime': {},  # Gives the time since a event was executed
                'pendingDeadline': {}  # The deadline until an event must be executed
                },
    'conditionsForDelays': {},
    'responseToDeadlines': {},
    'subprocesses': {},
    'nestings': {},
    'labels': set(),
    'labelMapping': {},
    'roles': set(),
    'roleAssignments': {},
    'readRoleAssignments': {}
}

In [33]:
debug = False

def encode(G):
    enc = {}
    for e in G['events']:
        enc[e] = set()
    for e in G['events']:
        for e_prime in G['events']:
            for rel in Relations:
                if e in G[rel.value] and e_prime in G[rel.value][e]:
                    if rel in [Relations.C,Relations.M]:
                        enc[e].add((e_prime,rel.value,'in'))
                    else:
                        enc[e].add((e_prime,rel.value,'out'))
                if e_prime in G[rel.value] and e in G[rel.value][e_prime]:
                    if rel in [Relations.C,Relations.M]:
                        enc[e].add((e_prime,rel.value,'out'))
                    else:
                        enc[e].add((e_prime,rel.value,'in'))
    return enc

def getOppositeRelDictStr(relStr, direction, event, nestingId):
    relationDictStrDel = (event,relStr,"out" if direction == "in" else "in")
    relationDictStrAdd = (nestingId,relStr,"out"  if direction == "in" else "in")

    return relationDictStrDel, relationDictStrAdd


def nest(G):
    nesting_map = {}
    nesting_ids = set()

    nest_id = 0

    enc = encode(G)
    events = G['events'].copy()

    # Should probably copy enc and events here as these are mutated
    def getLargestNesting(events, nesting_map, nesting_ids, nest_id, parentNesting=None):
      cands = {}
      for e in events:
          for j in events:
              arrow_s = frozenset(enc[e].intersection(enc[j]))
              if len(arrow_s)>0:
                  cands[arrow_s] = set([e, j])
      for arrow_s in cands.keys():
          for e in events:
              if len(arrow_s)>0 and arrow_s.issubset(enc[e]):
                  cands[arrow_s].add(e)

      best_score = 0
      best = None
      for arrow_s in cands.keys():
          cand_score = (len(cands[arrow_s])-1)*len(arrow_s)
          if cand_score>best_score:
              best_score = cand_score
              best = arrow_s

      if best_score == 0:
          return set(), nest_id
      if debug:
          print(f'[best] {cands[best]} {best} '
                f'\n [enc] \n {enc} \n '
                f'\n [cands] \n {cands} \n')

      nest_id += 1
      if debug:
        print("[nest id]", nest_id)
      nest_event = f'N{nest_id}'
      nesting_ids.add(nest_event)

      if (parentNesting != None):
        (parentId, nestingEvents) = parentNesting
        nestingEvents = nestingEvents.difference(cands[best])
        nestingEvents.add(nest_event)
        nesting_map[nest_event] = parentId

      enc[nest_event] = set(best)
      for e in cands[best]:
          nesting_map[e] = nest_event
          enc[e] = enc[e].difference(best)
          for (e_prime,rel,direction) in best:
              opRelDel, opRelAdd = getOppositeRelDictStr(rel,direction,e,nest_event)
              enc[e_prime].remove(opRelDel)
              enc[e_prime].add(opRelAdd)

      while True:
        foundNesting, nest_id = getLargestNesting(cands[best], nesting_map, nesting_ids, nest_id, parentNesting=(nest_event, cands[best]))
        if (len(foundNesting) == 0):
          break
              
      return cands[best], nest_id

    while True:
        # add a second while loop that recursively calls nest()
        foundNesting, nest_id = getLargestNesting(events, nesting_map, nesting_ids, nest_id,)
        if (len(foundNesting) == 0):
          break
        events = events.difference(foundNesting)
        if debug:
            print(f'[events] {events}')

    return enc, nesting_ids, nesting_map


enc, nesting_ids, nesting_map = nest(G)
print(nesting_ids,nesting_map)
print(enc)

{'N3', 'N1', 'N2'} {'F': 'N1', 'E': 'N2', 'D': 'N2', 'N2': 'N1', 'B': 'N3', 'A': 'N3'}
{'A': {('B', 'conditionsFor', 'in')}, 'G': {('N2', 'milestonesFor', 'in')}, 'F': set(), 'E': set(), 'B': {('C', 'conditionsFor', 'out'), ('A', 'conditionsFor', 'out')}, 'D': set(), 'C': {('B', 'conditionsFor', 'in'), ('N1', 'excludesTo', 'out')}, 'N1': {('N3', 'includesTo', 'in'), ('C', 'excludesTo', 'in')}, 'N2': {('G', 'milestonesFor', 'out')}, 'N3': {('N1', 'includesTo', 'out')}}


In [34]:
enc, nesting_ids, nesting_map = nest(sepsis_reference_dcr)
print(nesting_ids,nesting_map)
print(enc)

KeyError: ('Release E', 'excludesTo', 'out')

In [ ]:
n = Nesting()
n.create_encoding(G)
n.nest(G['events'])
n.get_nested_dcr_graph()

# Based on typescript code

In [ ]:
# import { DCRGraph, DCRSolutionsImport, RelationString, DirectlyFollows, DirectlyFollowsGraph, EventLog, EventMap, FuzzyDCRGraph, FuzzyEventMap, HiDCRGraph, Event, DCRSolutionsImportModel, FuzzyHiDCRGraph } from "../types";
# import fs from "fs";
# import { copyDCRGraph, copyEventMap, copyMarking, copySet, DCRSize, flipEventMap, intersect, printFull, setEqual, subset, unionGraphs } from "./utility";

In [ ]:
from copy import deepcopy

In [ ]:
RelationsDict = {}
ReturnNesting = {}
ReturnNesting['nestedGraph'] = {}
ReturnNesting['nestingIds'] = set()
ReturnNesting['nestingRelations'] = {}

DCRImportOptions = {
        'nestingIds': set(),
        'nestingRelations': {}
    }

def relationsDictToDCR(relationsDict):
    events = set(relationsDict.keys())
    graph = {
        'events': set(),
        'conditionsFor': {},
        'milestonesFor': {},
        'responseTo': {},
        'noResponseTo': {},
        'includesTo': {},
        'excludesTo': {},
        'marking': {'executed': set(),
                    'included': set(),
                    'pending': set()
                    }
    }

    def stringToRelation(strr):
        match strr:
            case "condition": return graph['conditionsFor']
            case "exclude": return graph['excludesTo']
            case "response": return graph['responseTo']
            case "include": return graph['includesTo']
            case "milestone": return graph['milestonesFor']

    def shouldAdd(str, direction):
        if str == "condition" or str == "milestone": 
            return direction == "in"
        else: 
            return direction == "out"

    for event in events:
        graph['conditionsFor'][event] = set()
        graph['responseTo'][event] = set()
        graph['includesTo'][event] = set()
        graph['excludesTo'][event] = set()
        graph['milestonesFor'][event] = set()

    for event, relations in relationsDict.items():
        for relation in relations:
            substrings = relation.split("_")
            if len(substrings < 3):
                print(f"[0] {substrings}")
                raise Exception("I didn't implement for this, easy fix though. ggez.")
            otherEvent = substrings[0] if len(substrings) == 3 else substrings.slice(0, -2).join("_")
            relStr = substrings[len(substrings) - 2]
            direction = substrings[len(substrings) - 1]

            if (shouldAdd(relStr, direction)):
                stringToRelation(relStr)[event].add(otherEvent)

    return graph

def getOppositeRelDictStr(relationDictStr, event, nestingId): #{ event: string, relationDictStrDel: string, relationDictStrAdd: string } => {
    substrings = relationDictStr.split("_")
    if len(substrings) > 3:
        print(f"[1] {substrings}")
        raise Exception("I didn't implement for this, easy fix though. ggez.")
    otherEvent = substrings[0] if len(substrings) == 3 else substrings.slice(0, -2).join("_")
    relStr = substrings[len(substrings) - 2]
    direction = substrings[len(substrings) - 1]

    relationDictStrDel = f'{event}_{relStr}_{"out" if direction == "in" else "in"}'
    relationDictStrAdd = f'{nestingId}_{relStr}_{"out"  if direction == "in" else "in"}'
    
    return otherEvent, relationDictStrDel, relationDictStrAdd

def setToStr(sett):
    return ''.join(sorted([s for s in sett]))

def flipEventMap(rel):
    flipped_rel = {event: set() for event in rel.keys()}
    for k,v in rel.items():
        for v0 in v:
            if v0 not in flipped_rel:
                flipped_rel[v0] = set([k])
            else:
                flipped_rel[v0].add(k)
    return flipped_rel

def getRelationsDict(graph):
    relationsDict = {}
    for event in graph['events']:
        relationsDict[event] = set()

    def addRelation(rel, name):
        flippedRel = flipEventMap(rel)
        # Ensure proper naming for flipped relations. Shouldn't matter in practice.
        d1, d2 = ("in", "out") if (name == "condition" or name == "milestone") else ("out", "in")

        for event in graph['events']:
            if event in rel:
                for otherEvent in rel[event]:
                    relationsDict[event].add(otherEvent + "_" + name + "_" + d1)
            if event in flippedRel:
                for otherEvent in flippedRel[event]:
                    relationsDict[event].add(otherEvent + "_" + name + "_" + d2)
        
    addRelation(graph['conditionsFor'], "condition")
    addRelation(graph['responseTo'], "response")
    addRelation(graph['excludesTo'], "exclude")
    addRelation(graph['includesTo'], "include")
    addRelation(graph['milestonesFor'], "milestone")

    return relationsDict

idCounter = 0

def getNestingId(idCounter): 
    idCounter += 1
    return f"Nesting{idCounter}"

def nestDCR(graph, minSharedRels, relationDict):

    nestingIds = set()
    nestingRelations = {}

    def findLargestNesting(events, parentNesting = None):      
        eventArr = list(events)
        possibleNestings = {}
        
        for i in range(0, len(eventArr)):
            for j in range(i,len(eventArr)):
                e1 = eventArr[i]
                e2 = eventArr[j]
                shared = relationDict[e1].intersection(relationDict[e2])
                possibleNestings[setToStr(shared)] = { 'nestingEvents':set([e1, e2]), 'sharedRels': shared }

        for possibleNestingId, nesting in possibleNestings.items():
            for event in events:
                if nesting['sharedRels'].issubset(relationDict[event]):
                    nesting['nestingEvents'].add(event)

        bestNesting = { 'nestingEvents': set(), 'sharedRels': set() }
        max = 0
        for possibleNestingId, nesting in possibleNestings.items():
            
            metric = (len(nesting['nestingEvents']) - 1) * len(nesting['sharedRels'])
            if (metric > max):
                max = metric
                bestNesting = nesting

        retval = []#{ nestingEvents: Set<Event>, sharedRels: Set<string> }> = [];
        if len(bestNesting['nestingEvents']) > 1 and len(bestNesting['sharedRels']) >= minSharedRels:

            nestingId = getNestingId(idCounter)

            nestingIds.add(nestingId)

            if parentNesting:
                parentNesting['events'].difference(bestNesting['nestingEvents'])
                parentNesting['events'].add(nestingId)
                nestingRelations[nestingId] = parentNesting['id']
            
            for event in bestNesting['nestingEvents']:
                nestingRelations[event] = nestingId

            for nestedEvent in bestNesting['nestingEvents']:
                relationDict[nestedEvent].difference(bestNesting['sharedRels'])
                for sharedRelStr in bestNesting['sharedRels']:
                    event, relationDictStrDel, relationDictStrAdd = getOppositeRelDictStr(sharedRelStr, nestedEvent, nestingId)
                    relationDict[event].add(relationDictStrAdd)
                    relationDict[event].discard(relationDictStrDel)

            relationDict[nestingId] = deepcopy(bestNesting['sharedRels'])

            retval = []#[nestingEvents: bestNesting.nestingEvents, sharedRels: bestNesting.sharedRels }];
            found = True
            while (found):
                largestNesting = findLargestNesting(bestNesting['nestingEvents'], { 'id': nestingId, 'events': bestNesting['nestingEvents'] })
                if len(largestNesting) != 0:
                    retval.extend(largestNesting)
                else: 
                    found = False

        return retval

    nestingsFound = 0
    nestingsArr = []#<{ nestingEvents: Set<Event>, sharedRels: Set<string> }> = [];
    events = deepcopy(graph['events'])
    while True:
        partialNestingArr = findLargestNesting(events)
        nestingsFound = len(partialNestingArr)
        nestingsArr.extend(partialNestingArr)
        if len(partialNestingArr) > 0: 
            events.difference(partialNestingArr[0]['nestingEvents'])
        if not nestingsFound > 0:
            break

    return relationsDictToDCR(relationDict), deepcopy(graph['events']).union(nestingIds), nestingIds, nestingRelations

In [ ]:
graph = {
    'events': set(['A','B','C','D','E','F']),
    'conditionsFor': {'A':{'B'},'C':{'B'}},
    'milestonesFor': {},
    'responseTo': {},
    'noResponseTo': {},
    'includesTo': {'A':{'D','E','F'},'B':{'D','E','F'}},
    'excludesTo': {'C':{'D','E','F'}},
    'marking': {'executed': set(),
                'included': set(['A','B','C','D','E','F']),
                'pending': set(),
        }
    }    
nestDCR(graph,minSharedRels=1, relationDict=getRelationsDict(graph))

# Other clustering (from networkx)

In [ ]:
nxG = dcr_to_networkx(G)
nx.draw_circular(nxG, with_labels=True)
for n in nxG.nodes:
    print(n)
    print([s for s in nxG.successors(n)])
    print([p for p in nxG.predecessors(n)])
nx.clustering(nxG)